#### Min-Cut: Karger's Algorithm

Implementation of Karger's algorithm (Contraction algorithm) for finding global minimum cut of a connected undirected graph.

In [39]:
import random
random.seed(0)

In [40]:
# contract edge (u,v) and return the adjacenecy list of the resulting graph
def contract_edge(u,v, adjacency_list, supernode_dict, new_supernode_id):
    assert u in adjacency_list and v in adjacency_list, "u and v must be in the adjacency list"

    # merge u and v into a new super node
    w = new_supernode_id
    supernode_dict[w] = supernode_dict[u] + supernode_dict[v]
    del supernode_dict[u]
    del supernode_dict[v]

    # update adjacent list
    adjacency_list[w] = []
    # connect w with edges incident on u (exclude self-loop)
    for node in adjacency_list[u]:
        if node != v:
            adjacency_list[w].append(node)
            adjacency_list[node].remove(u)
            adjacency_list[node].append(w)

    # connect w with edges incident on v (exclude self-loop)
    for node in adjacency_list[v]:
        if node != u:
            adjacency_list[w].append(node)
            adjacency_list[node].remove(v)
            adjacency_list[node].append(w)

    # remove u and v from the adjacency list
    del adjacency_list[u]
    del adjacency_list[v]        


    return adjacency_list, supernode_dict, new_supernode_id+1   


def contraction_algorithm(adjacency_list, verbose=False):
    # make a copy of the adjacency list
    adjacency_list_original = {node: list(neighbors) for node, neighbors in adjacency_list.items()}
    # create supernode dictionary, each supernode is a list/set of nodes, initially each node is a supernode,
    supernode_dict = {}
    for node in adjacency_list:
        supernode_dict[node] = [node]

    if verbose:
        print(f"G = {adjacency_list}")
        print(f"initial supernodes: {supernode_dict}")

    new_supernode_id = len(adjacency_list)   

    # randomly select an edge (u,v) to contract
    while len(adjacency_list) > 2:
        # randomly select an edge (u,v) to contract
        u = random.choice(list(adjacency_list.keys()))
        v = random.choice(adjacency_list[u])
        # contract the edge
        adjacency_list, supernode_dict, new_supernode_id = contract_edge(u,v, adjacency_list, supernode_dict, new_supernode_id)
        if verbose:
            print(f"\ncontracting edge ({u},{v})")
            print(f"supernodes: {supernode_dict}")
            print(f"updated G: {adjacency_list}")

    # extract all edges in cut(A,B)
    supernode_sets = list(supernode_dict.values())
    A = supernode_sets[0]
    B = supernode_sets[1]
    if verbose:
        print(f"\nA = {A}, B = {B}")

    mincut = []
    for u in A:
        for v in adjacency_list_original[u]:
            if v in B:
                mincut.append((u, v))

    return mincut
     

In [50]:
adjacency_list = {0:[1,2,3], 1:[0,2], 2:[0,1,3], 3:[0,2]}

mincut = contraction_algorithm(adjacency_list, verbose=True)
mincut

G = {0: [1, 2, 3], 1: [0, 2], 2: [0, 1, 3], 3: [0, 2]}
initial supernodes: {0: [0], 1: [1], 2: [2], 3: [3]}

contracting edge (0,3)
supernodes: {1: [1], 2: [2], 4: [0, 3]}
updated G: {1: [2, 4], 2: [1, 4, 4], 4: [1, 2, 2]}

contracting edge (1,2)
supernodes: {4: [0, 3], 5: [1, 2]}
updated G: {4: [5, 5, 5], 5: [4, 4, 4]}

A = [0, 3], B = [1, 2]


[(0, 1), (0, 2), (3, 2)]